In [1]:
from ToT.util import query_local_llm
from ToT.Tree import ThoughtNode

ModuleNotFoundError: No module named 'util'

In [144]:
test_question = "Let's play challenge 24! The goal of the game is to use arithmetic operations (+, -, x, /) to find a combination of the given numbers that equal 24. You must use all of the numbers and you can only use each number once. Here is your problem: (2, 6, 1, 6)."
test_thought = '''2 * 6 + 1 * 6 = 24

The solution to this problem involves multiplying 2 by 6 to get 12, then multiplying 1 by 6 to get 6. Adding these together results in the target number of 24.'''

t, ct = query_local_llm([test_question])

In [145]:
print(t)

2 * (6 + 1 - 6)

The expression simplifies to 0, which does not equal 24. Therefore, it's not possible to find a combination of the numbers that equals 24 using each number only once and the operations of addition (+), subtraction (-), multiplication (x), and division (/).


In [138]:
test_node = ThoughtNode(test_thought, test_question)

In [139]:
print(test_node.get_thought_process())

2 * 6 + 1 + 6 = 24

So, the combination is: (2 * 6) + 1 + 6


In [140]:
test_node.generate_children(test_question)

In [141]:
print(len(test_node.children))

0


In [142]:
for n in test_node.children:
    print(n.get_thought_process())

In [ ]:
problem = input('Enter a Query: ')
tree = ThoughtTree(problem)
solution = tree.search()

print("Problem:\n", problem, '\n')
print("\nSolution path:")
for i, thought in enumerate(solution):
    print(f"{i+1}:\n{thought}\n")